In [0]:
%sql
CREATE VOLUME IF NOT EXISTS users.alex_vanadio.realtime_leaderboard;

In [0]:
import json, math, pathlib, requests

In [0]:
# Notebook cell: fetch Data Dragon items and write JSONL
# Configure these:
VERSION = "latest"     # or like "14.20.1"
LOCALE = "en_US"       # e.g., en_US, en_GB, ko_KR
OUT_JSONL = "/Volumes/users/alex_vanadio/realtime_leaderboard/ddragon_items.json" 

VERSIONS_URL = "https://ddragon.leagueoflegends.com/api/versions.json"
ITEMS_URL_TMPL = "https://ddragon.leagueoflegends.com/cdn/{version}/data/{locale}/item.json"

def fetch_versions():
    r = requests.get(VERSIONS_URL, timeout=20)
    r.raise_for_status()
    return r.json()

def fetch_items(version: str, locale: str):
    url = ITEMS_URL_TMPL.format(version=version, locale=locale)
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def flatten_items(dd_json: dict, version: str, locale: str):
    data = dd_json.get("data", {}) or {}
    rows = []
    for sid, meta in data.items():
        try:
            item_id = int(sid)
        except Exception:
            continue  # skip non-numeric keys just in case

        gold = meta.get("gold", {}) or {}
        rows.append({
            "dd_version": version,
            "locale": locale,
            "itemId": item_id,
            "name": meta.get("name"),
            "plaintext": meta.get("plaintext"),
            "description": meta.get("description"),
            "totalCost": int(gold.get("total") or 0),
            "baseCost": int(gold.get("base") or 0),
            "sellGold": int(gold.get("sell") or 0),
            "tags": meta.get("tags", []),
            "depth": meta.get("depth"),
            "stacks": meta.get("stacks"),
            "rarityMythic": bool(meta.get("rarityMythic", False)),
            "consumable": bool(meta.get("consumable", False)),
            "inStore": meta.get("inStore", True),
            "requiredChampion": meta.get("requiredChampion"),
            "requiredAlly": meta.get("requiredAlly"),
            "fromIds": [int(x) for x in meta.get("from", []) if str(x).isdigit()],
            "intoIds": [int(x) for x in meta.get("into", []) if str(x).isdigit()],
            # keep only maps where the item is available
            "maps": sorted([int(k) for k, v in (meta.get("maps", {}) or {}).items() if v]),
        })
    rows.sort(key=lambda r: r["itemId"])
    return rows

# Resolve version
version = VERSION
if version.lower() == "latest":
    versions = fetch_versions()
    assert versions, "Could not fetch DDragon versions."
    version = versions[0]

# Fetch, flatten, write JSONL
dd = fetch_items(version, LOCALE)
rows = flatten_items(dd, version, LOCALE)

out_path = pathlib.Path(OUT_JSONL)
out_path.parent.mkdir(parents=True, exist_ok=True)

with out_path.open("w", encoding="utf-8") as f:
    for rec in rows:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"Wrote {len(rows)} items to {out_path} (version={version}, locale={LOCALE})")

Wrote 635 items to /Volumes/users/alex_vanadio/test_volume/ddragon_items.json (version=15.17.1, locale=en_US)


In [0]:
spark.read.json(OUT_JSONL).display()

baseCost,consumable,dd_version,depth,description,fromIds,inStore,intoIds,itemId,locale,maps,name,plaintext,rarityMythic,requiredAlly,requiredChampion,sellGold,stacks,tags,totalCost
300,false,15.17.1,null,25 Move Speed,List(),true,"List(3005, 3047, 3006, 3009, 3010, 3020, 3111, 3117, 3158)",1001,en_US,"List(11, 12, 21, 35)",Boots,Slightly increases Move Speed,false,null,null,210,null,List(Boots),300
200,false,15.17.1,null,50% Base Mana Regen,List(),true,"List(3114, 3012, 4642)",1004,en_US,"List(11, 12, 21, 35)",Faerie Charm,Slightly increases Mana Regen,false,null,null,140,null,List(ManaRegen),200
300,false,15.17.1,null,100% Base Health Regen,List(),true,"List(3109, 3211, 323109, 3801)",1006,en_US,"List(11, 12, 21, 35)",Rejuvenation Bead,Slightly increases Health Regen,false,null,null,120,null,List(HealthRegen),300
500,false,15.17.1,2,350 Health,List(1028),true,"List(4637, 3084, 2502, 3083, 3116, 3119, 3143, 323119, 3748, 6665, 6609, 6667, 8001)",1011,en_US,"List(11, 12, 21, 35)",Giant's Belt,Greatly increases Health,false,null,null,630,null,List(Health),900
600,false,15.17.1,null,15% Critical Strike Chance,List(),true,"List(667666, 3031, 3086, 6670, 3033, 3508, 3095, 6676)",1018,en_US,"List(11, 12, 21, 35)",Cloak of Agility,Increases critical strike chance,false,null,null,420,null,List(CriticalStrike),600
850,false,15.17.1,null,45 Ability Power,List(),true,"List(326621, 4637, 4628, 3135, 3115, 3100, 3116, 3118, 3165, 326657, 6621, 6657)",1026,en_US,"List(11, 12, 21, 35)",Blasting Wand,Moderately increases Ability Power,false,null,null,595,null,List(SpellDamage),850
300,false,15.17.1,null,300 Mana,List(),true,"List(3024, 3802, 3803)",1027,en_US,"List(11, 12, 21, 35)",Sapphire Crystal,Increases Mana,false,null,null,210,null,List(Mana),300
400,false,15.17.1,null,150 Health,List(),true,"List(3742, 1011, 2021, 323222, 323075, 326617, 323107, 3012, 3147, 3023, 3044, 3053, 3066, 3067, 3075, 3211, 3152, 3161, 323050, 3801, 3803, 4401, 4635, 6035, 6610, 6660, 6662)",1028,en_US,"List(11, 12, 21, 35)",Ruby Crystal,Increases Health,false,null,null,280,null,List(Health),400
300,false,15.17.1,null,15 Armor,List(),true,"List(2421, 2420, 1031, 323190, 2019, 3193, 3047, 3024, 3050, 3023, 3076, 3082, 3105, 3190)",1029,en_US,"List(11, 12, 21, 35)",Cloth Armor,Slightly increases Armor,false,null,null,210,null,List(Armor),300
500,false,15.17.1,2,40 Armor,List(1029),true,"List(3742, 3068, 323002, 3109, 323075, 3002, 3075, 323109, 6665, 6662)",1031,en_US,"List(11, 12, 21, 35)",Chain Vest,Greatly increases Armor,false,null,null,560,null,List(Armor),800


In [0]:
spark.read.json(OUT_JSONL).write.saveAsTable("users.alex_vanadio.ddragon_items", mode="overwrite")